In [1]:
#Import the different packages needed
from IPython.display import display
import pandas as pd
#Changes the default feature when displaying
pd.options.display.max_columns = None # Display all columns of a dataframe
pd.options.display.max_rows = 700
from pprint import pprint
import re
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
#Get our webdriver
chromeOptions = webdriver.ChromeOptions()
#it is way to add options

path = "C:/Users/User/Documents/GitHub/Films_DM_2023/chromedriver_win32" #the path

driver = webdriver.Chrome(executable_path=os.path.join(path, 'chromedriver'))


C:\Users\User\AppData\Local\Temp\ipykernel_49796\2095578362.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=os.path.join(path, 'chromedriver'))


In [3]:
driver.get("https://www.allocine.fr/")
time.sleep(1) # Let the user actually see something!

In [4]:
#Accepter les cookies pour pouvoir passer à la suite
cookies = driver.find_element(By.CLASS_NAME, "jad_cmp_paywall_button-cookies").click()
time.sleep(2)

In [6]:
#webscraping in itself

#1) initialize the different lists needed to store information
films =[]
films_addedinfo=[]
real_info=[]
act1_info=[]
act2_info=[]

#2) create a function webscraping the page
def my_function():
    
    #First create the list that will be used to navigate between the different parts
    #If we put them before the function, they are not reinitialized each time which is problematic
    links=[]
    links_real=[]
    links_actor1=[]
    links_actor2=[]
    
    #First, separate each film (=> get the "box" about each film in the list)
    for each_fiche in driver.find_elements(By.CLASS_NAME,'mdl'):
        
        #titles
        titre=each_fiche.find_element(By.CLASS_NAME, "meta-title-link").text
        
        #release date (or date_sortie in French)
        date_sortie= each_fiche.find_element(By.CLASS_NAME, "date").text
        
        #durée (length)
        duree =each_fiche.find_element(By.CLASS_NAME, "meta-body-info").text.split('/')[1]
    
        #Genres
        info =each_fiche.find_element(By.CLASS_NAME, "meta-body-info")
        genres= info.find_elements(By.CLASS_NAME, "xXx")
        
        #Initiate the variebles (otherwise, pb with local and global variables)
        genre_1="NA"
        genre_2="NA"
        genre_3="NA"
        
        #genre1
        genre_1 = genres[0].text
        #Genre 2 et 3
        if len(genres)>=2:
            genre_2= genres[1].text
            if len(genres)==3:
                genre_3=genres[2].text
        
        #Directors
        #realisateur= each_fiche.find_element(By.CLASS_NAME, "blue-link").text
        direction =each_fiche.find_element(By.CLASS_NAME, "meta-body-direction")
        realisateurs= direction.find_elements(By.CSS_SELECTOR, "a")
    
        #Get the link to the first realisator page to get info if he got awards later
        link_real= realisateurs[0].get_attribute('href')
        links_real.append(link_real)
        
    
        #realisateur1
        real_1 = realisateurs[0].text
        #real 2
        if len(realisateurs)>=2:
            real_2= realisateurs[1].text
        else:
            real_2="NA"
    
        #Actors
        casting =each_fiche.find_element(By.CLASS_NAME, "meta-body-actor")
        actors= casting.find_elements(By.CSS_SELECTOR, "a")
    
        #Get the link to the actors 1
        link_actor1= actors[0].get_attribute('href')
        links_actor1.append(link_actor1)
    
        #Get the link to the actors 2
        link_actor2= actors[1].get_attribute('href')
        links_actor2.append(link_actor2)
    
        #actor1
        actor_1 = actors[0].text
        #real 2
        if len(actors)>=2:
            actor_2= actors[1].text
            if len(actors)==3:
                actor_3=actors[2].text
            else:
                actor_3="NA"
        else:
            actor_2="NA"
        
        #Ratings
        #First initiate them in NA if does not exist
        note_spectateur="NA"
        note_presse="NA"
        #Look for the notes
        if each_fiche.find_elements(By.CLASS_NAME, "rating-holder"):
            rating = each_fiche.find_element(By.CLASS_NAME, "rating-holder")
            ratings=rating.find_elements(By.CLASS_NAME,'rating-item-content')
            #For the first one
            rating_type_1=ratings[0].find_element(By.CLASS_NAME,'rating-title').text
            if(rating_type_1== "PRESSE"):
                note_presse= ratings[0].find_element(By.CLASS_NAME, 'stareval-note').text
            elif(rating_type_1== "SPECTATEURS"):
                note_spectateur= ratings[0].find_element(By.CLASS_NAME, 'stareval-note').text 
        
            #For the second one
            if ratings[1].find_element(By.CLASS_NAME,'rating-title').text:
                rating_type_2=ratings[1].find_element(By.CLASS_NAME,'rating-title').text
                if(rating_type_2== "SPECTATEURS"):
                    note_spectateur= ratings[1].find_element(By.CLASS_NAME, 'stareval-note').text 
                elif(rating_type_2== "PRESSE"): #sometimes it is not in the same order
                    note_presse= ratings[1].find_element(By.CLASS_NAME, 'stareval-note').text
    
        #Now we are done with the information we could find on the first page => click on the link for each film and go see more info
        #Problem, if come back to main page, stale elements => work around it by first creating a vector with all the links and visiting at the end without coming back to the main page
        link= each_fiche.find_element(By.CLASS_NAME, "meta-title-link").get_attribute('href')
        links.append(link)
    

        films.append({
                "Titre": titre,
                "date_sortie" : date_sortie,
                "duree": duree,
                "genre1": genre_1,
                "genre2": genre_2,
                "genre3": genre_3,
                "réalisateur_1": real_1,
                "réalisateur_2": real_2,
                "actor_1": actor_1,
                "actor 2": actor_2,
                "actor 3": actor_3,
                "note_presse": note_presse,
                "note spectators": note_spectateur,
                "link":link,
                "link realisator": link_real,
            })
    
    #Get info on second page (more info on films)
    for link in links:
        driver.get(link)
        time.sleep(2)
        infos = driver.find_element(By.CLASS_NAME,'meta-body')
        #Scenaristes
        scenar = infos.find_element(By.XPATH,'div[3]')
        scenarists = scenar.find_elements(By.CSS_SELECTOR,'a')
        scenar_1 = scenarists[0].text
    
        #Technical informations
        techni= driver.find_element(By.CLASS_NAME, 'ovw-technical')
        #nationality
        natio=techni.find_element(By.CLASS_NAME, 'nationality').text
    
        #Distributor
        nom=techni.find_element(By.XPATH,'div[3]/span').text
        if(nom == "Distributeur"):
            if techni.find_elements(By.XPATH,'div[3]/a'):
                distributor = techni.find_element(By.XPATH,'div[3]/a').text
            else: 
                distributor="NA"
            
        #Palmares
    
        #Attention, this does not exist for all the films!!!
    
        #If it exists
        if(techni.find_element(By.XPATH,'div[4]/span').text == "Récompense" or techni.find_element(By.XPATH,'div[4]/span').text == "Récompenses" ):
            #if techni.find_element(By.XPATH,'div[4]/a'):
            recompense = techni.find_element(By.XPATH,'div[4]/a').text
            recompense=str(re.sub("[^0-9^]"," ",recompense))
            #As if there is a recompense, there is one row more in the table with infos, it changes our div => put for the ones with recompenses here
            budget=0
        
            #Box office
            exist=techni.find_element(By.XPATH,'div[11]/span').text
            if(exist == "Box Office France"):
                boxeoffice=techni.find_element(By.XPATH,'div[11]/a').text
            elif (exist == "Budget"): #As there is not always a raw boxoffice
                budget=techni.find_element(By.XPATH,'div[11]/span[2]').text
            
            #Budget
            exist2=techni.find_element(By.XPATH,'div[12]/span').text
            if(exist2 == "Budget"):
                budget=techni.find_element(By.XPATH,'div[12]/span[2]').text
            elif (exist2 == "Langues"): #As there is not always a raw boxoffice and this repercutes
                language=techni.find_element(By.XPATH,'div[12]/span[2]').text
            
            #Language
            exist6=techni.find_element(By.XPATH,'div[13]/span').text
            if(exist6 == "Langues"):
                language=techni.find_element(By.XPATH,'div[13]/span[2]').text
        
        
        else:  #If the film has no recompense field 
            recompense= "0"
        
            #Budget
            budget="NA"
            boxeoffice="NA"
    
            #Box office
            exist3=techni.find_element(By.XPATH,'div[10]/span').text
            if(exist3 == "Box Office France"):
                boxeoffice=techni.find_element(By.XPATH,'div[10]/a').text
            elif (exist3 == "Budget"):
                budget=techni.find_element(By.XPATH,'div[10]/span[2]').text
            
            #Budget
            exist4=techni.find_element(By.XPATH,'div[11]/span').text
            if(exist4 == "Budget"):
                budget=techni.find_element(By.XPATH,'div[11]/span[2]').text
            elif (exist4 == "Langues"):
                language=techni.find_element(By.XPATH,'div[11]/span[2]').text
            
            #Language
            exist5=techni.find_element(By.XPATH,'div[12]/span').text
            if(exist5 == "Langues"):
                language=techni.find_element(By.XPATH,'div[12]/span[2]').text
  

        films_addedinfo.append({
                "Scenar": scenar_1,
                "nationality": natio,
                "distributor": distributor,
                "nominations": recompense,
                "budget": budget,
                "language": language,
                "box-office":boxeoffice,
            })
    
    
    #Get more info on the realisator

    for real in links_real:
        driver.get(real)
        time.sleep(2)
        if driver.find_elements(By.CLASS_NAME,'stats-numbers-row'):
            real_infos = driver.find_element(By.CLASS_NAME,'stats-numbers-row')
            if real_infos.find_elements(By.CSS_SELECTOR,'a'):
                award_real = real_infos.find_element(By.XPATH,"//a/div[@class='stats-number']").text
            else: 
                award_real=0
        else:
            award_real="NA"
        real_info.append({"real awards":award_real,})
    
    
    #Get more info on the actors 1
    for actor1 in links_actor1:
        driver.get(actor1)
        time.sleep(2)
        if driver.find_elements(By.CLASS_NAME,'stats-numbers-row'):
            act1_infos = driver.find_element(By.CLASS_NAME,'stats-numbers-row')
            if act1_infos.find_elements(By.CSS_SELECTOR,'a'):
                award_act1 = act1_infos.find_element(By.XPATH,"//a/div[@class='stats-number']").text
            else: 
                award_act1=0
        else: 
            award_act1="NA"
        act1_info.append({"first actor awards":award_act1,})

    #Get more info on the actors 2
    for actor2 in links_actor2:
        driver.get(actor2)
        time.sleep(1.5)
        if driver.find_elements(By.CLASS_NAME,'stats-numbers-row'):
            act2_infos = driver.find_element(By.CLASS_NAME,'stats-numbers-row')
            if act2_infos.find_elements(By.CSS_SELECTOR,'a'):
                award_act2 = act2_infos.find_element(By.XPATH,"//a/div[@class='stats-number']").text
            else: 
                award_act2=0
        else:
            award_act2="NA"
        act2_info.append({"Second actor awards":award_act2,})

#Get it for different pages


for page in range(1,3):
    raw_url = "https://www.allocine.fr/films/alphabetique/?page={}"
    driver.get(raw_url.format(str(page)))
    time.sleep(1) # Let the user actually see something!
    time.sleep(2)
    my_function()
    
    
#dataframe
films_data = pd.DataFrame(films)
films_added= pd.DataFrame(films_addedinfo)
real_info=pd.DataFrame(real_info)
act1_info=pd.DataFrame(act1_info)
act2_info=pd.DataFrame(act2_info)

#Concatenate the different df to have one final one
Movies = pd.concat([films_data, films_added,real_info,act1_info,act2_info], axis=1)
Movies

#Pass it in CSV to have a file to use later (indeed, selenium is moving so we want to have one for once and all)

movies_bd = df.to_csv(Movies, encoding='utf-8', index=False)

,Titre,date_sortie,duree,genre1,genre2,genre3,réalisateur_1,réalisateur_2,actor_1,actor 2,actor 3,note_presse,note spectators,link,link realisator,Scenar,nationality,distributor,nominations,budget,language,box-office,real awards,first actor awards,Second actor awards
0,« Peu m'importe si l'Histoire nous considère c...,20 février 2019,2h 19min,Comédie,Drame,NA,Radu Jude,NA,Ioana Iacob,Alexandru Dabija,Alexandru Bogdan,"3,8","3,3",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Radu Jude,Roumanie,Météore Films,0,-,Roumain,901 entrées,6,0,0
1,# Pire soirée,2 août 2017,1h 41min,Comédie,NA,NA,Lucia Aniello,NA,Scarlett Johansson,Kate McKinnon,Jillian Bell,"1,6","1,8",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Lucia Aniello,U.S.A.,Sony Pictures Releasing France,0,"$20,000,000",Anglais,97 842 entrées,2,3,1
2,#Alive,23 septembre 2021,1h 38min,Action,Drame,Epouvante-horreur,Hyung-cho Il,NA,Ah-In Yoo,Shin-Hye Park,Hyun-Wook Lee,NA,"3,0",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Hyung-cho Il,Corée du Sud,Netflix,0,-,Coréen,NA,NA,0,NA
3,#Amour et déconnexion,31 mai 2021,1h 30min,Famille,NA,NA,Bradford May,NA,Mischa Barton,Sean Faris,B.J. Mitchell,NA,"3,0",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Mischa Barton,U.S.A.,NA,0,-,Anglais,NA,0,0,0
4,#Chef,29 octobre 2014,1h 54min,Comédie,Drame,NA,Jon Favreau,NA,Jon Favreau,John Leguizamo,Sofia Vergara,"2,8","3,6",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Jon Favreau,U.S.A.,Sony Pictures Releasing France,1,-,Anglais,15 801 entrées,6,6,4
5,#FriendButMarried,14 novembre 2022,1h 42min,Biopic,Comédie dramatique,Romance,Rako Prijanto,NA,Adipati Dolken,Vanesha Prescilla,Refal Hady,NA,NA,https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Adipati Dolken,Indonésie,NA,0,-,Indonésien,NA,NA,0,0
6,#Home,18 août 2021,2h 38min,Drame,Famille,NA,Rojin Thomas,NA,Indrans,Sreenath Bhasi,Nalsen K. Gafoor,NA,NA,https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Rojin Thomas,Inde,NA,0,-,Malais,NA,NA,0,NA
7,#Horror,27 janvier 2019,1h 30min,Drame,Epouvante-horreur,Thriller,Tara Subkoff,NA,Mina Sundwall,Bridget McGarry,Haley Murphy,NA,"1,1",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Tara Subkoff,U.S.A.,NA,0,-,Anglais,NA,0,0,0
8,#Jesuislà,5 février 2020,1h 38min,Comédie,Romance,NA,Eric Lartigau,NA,Alain Chabat,Doona Bae,Blanche Gardin,"3,0","2,4",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Eric Lartigau,France,Gaumont Distribution,0,-,"Coréen, Français",193 851 entrées,13,2,0
9,#Moscou-Royan,21 novembre 2018,1h 23min,Comédie,NA,NA,Elena Cosson Kizilova,NA,Elena Cosson Kizilova,Donia Eden,Manuel Sorroche,NA,"3,1",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Elena Cosson Kizilova,France,Leva Productions,0,-,Français,NA,NA,NA,0
